<a href="https://colab.research.google.com/github/Rkk-i8/Canalicular_excision/blob/main/Canalicular_excision1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ❶ Google Drive をマウント
from google.colab import drive
drive.mount('/content/drive')

# ❷ 必要なライブラリのインポート
import pandas as pd
from scipy.stats import mannwhitneyu

# ❸ Excel ファイルの読み込み
file_path = "/content/drive/Shareddrives/岩崎Dr_IgG4 deulk/涙小管切断術/Canalicular Occlusion.xlsx"
df_all = pd.read_excel(file_path, sheet_name='all')

# ❹ Fluo_5area_12M による群分け
# 良好群：Fluo_5area_12M ≦ 3、遷延群：Fluo_5area_12M ≧ 4
df_all['group'] = df_all['Fluo_5area_12M'].apply(lambda x: '遷延群' if x >= 4 else ('良好群' if x <= 3 else None))

# ❺ 群ごとの眼数を表示
group_counts = df_all['group'].value_counts()
print("■ 群ごとの眼数")
print(group_counts, "\n")

# ❻ 対象とする4項目
target_cols = ['Schirmer 1_pre', 'Fluo_5area_pre', 'TM_mm_pre', 'TM_mm_12M']

# ❼ 中央値 [IQR] の計算関数
def median_iqr(series):
    q1 = series.quantile(0.25)
    median = series.median()
    q3 = series.quantile(0.75)
    return f"{median:.3f} [{q1:.3f}–{q3:.3f}]"

# ❽ 良好群・遷延群それぞれの中央値 [IQR]
formatted_stats = {}
for group in ['良好群', '遷延群']:
    group_data = df_all[df_all['group'] == group]
    formatted_stats[group] = {col: median_iqr(group_data[col].dropna()) for col in target_cols}

# ❾ 表形式で表示
median_iqr_table = pd.DataFrame(formatted_stats).T[target_cols]
print("■ 各群の中央値 [IQR]")
print(median_iqr_table.to_string(), "\n")

# ➓ Mann-Whitney U検定による2群間比較
stat_results = {}
for col in target_cols:
    group1 = df_all[df_all['group'] == '良好群'][col].dropna()
    group2 = df_all[df_all['group'] == '遷延群'][col].dropna()
    stat, p_value = mannwhitneyu(group1, group2, alternative='two-sided')
    stat_results[col] = {'U Statistic': stat, 'p-value': p_value}

# 結果をデータフレーム化して表示
stat_results_df = pd.DataFrame(stat_results).T
print("■ Mann-Whitney U検定の結果")
print(stat_results_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
■ 群ごとの眼数
group
遷延群    20
良好群    14
Name: count, dtype: int64 

■ 各群の中央値 [IQR]
          Schirmer 1_pre         Fluo_5area_pre            TM_mm_pre            TM_mm_12M
良好群  1.500 [1.000–3.000]    7.500 [3.500–8.750]  0.131 [0.098–0.171]  0.232 [0.185–0.329]
遷延群  2.500 [2.000–4.000]  11.000 [8.000–13.000]  0.129 [0.111–0.144]  0.287 [0.150–0.331] 

■ Mann-Whitney U検定の結果
                U Statistic   p-value
Schirmer 1_pre        109.5  0.286789
Fluo_5area_pre         46.5  0.001067
TM_mm_pre             142.0  0.958072
TM_mm_12M             142.5  0.944180


In [3]:
# ❶ Google Drive をマウント
from google.colab import drive
drive.mount('/content/drive')

# ❷ ライブラリのインポート
import pandas as pd
from scipy.stats import wilcoxon

# ❸ Excel ファイルの読み込み
file_path = "/content/drive/Shareddrives/岩崎Dr_IgG4 deulk/涙小管切断術/Canalicular Occlusion.xlsx"
df_all = pd.read_excel(file_path, sheet_name='all')

# ❹ 対象とする列
columns_of_interest = [
    'Schirmer 1_pre', 'Fluo_5area_pre', 'Fluo_5area_3M', 'Fluo_5area_12M',
    'Observation periods', 'Recurrent periods',
    'TM_mm_pre', 'TM_mm_3M', 'TM_mm_12M'
]

# ❺ 中央値と四分位範囲（Q1–Q3）の算出
summary_stats = df_all[columns_of_interest].describe(percentiles=[.25, .5, .75]).T
summary_stats = summary_stats[['25%', '50%', '75%']]
summary_stats.columns = ['Q1', 'Median', 'Q3']

print("■ 中央値 [IQR]")
print(summary_stats.to_string(), "\n")

# ❻ Wilcoxon符号付順位検定（術前 vs 術後ペアデータ比較）
comparison_pairs = [
    ('Fluo_5area_pre', 'Fluo_5area_3M'),
    ('Fluo_5area_pre', 'Fluo_5area_12M'),
    ('TM_mm_pre', 'TM_mm_3M'),
    ('TM_mm_pre', 'TM_mm_12M')
]

wilcoxon_results = {}
for pre, post in comparison_pairs:
    paired_data = df_all[[pre, post]].dropna()
    stat, p_value = wilcoxon(paired_data[pre], paired_data[post])
    wilcoxon_results[f"{pre} vs {post}"] = {
        'Wilcoxon Statistic': stat,
        'p-value': p_value
    }

# 結果をデータフレームとして表示
wilcoxon_df = pd.DataFrame(wilcoxon_results).T
print("■ Wilcoxon 検定結果（術前 vs 術後）")
print(wilcoxon_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
■ 中央値 [IQR]
                            Q1     Median         Q3
Schirmer 1_pre        1.000000   2.000000   3.750000
Fluo_5area_pre        7.000000   9.000000  11.750000
Fluo_5area_3M         2.000000   3.000000   6.000000
Fluo_5area_12M        2.250000   4.000000   8.500000
Observation periods  24.500000  29.500000  43.000000
Recurrent periods     2.500000   4.000000   5.750000
TM_mm_pre             0.098531   0.129426   0.154080
TM_mm_3M              0.210000   0.295161   0.325096
TM_mm_12M             0.179487   0.265064   0.335503 

■ Wilcoxon 検定結果（術前 vs 術後）
                                  Wilcoxon Statistic       p-value
Fluo_5area_pre vs Fluo_5area_3M                 53.5  8.075045e-05
Fluo_5area_pre vs Fluo_5area_12M                49.5  5.669204e-05
TM_mm_pre vs TM_mm_3M                            7.0  6.803144e-07
TM_mm_pre vs TM_mm_12M           